In [63]:
from datetime import datetime

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


tqdm.pandas()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 40)

In [10]:
df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)
df.head()

C:\Users\Martin\AppData\Local\Temp\ipykernel_23524\3437320206.py:1: DtypeWarning: Columns (65,76,91,154,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,GBH,GBD,GBA,IWH,IWD,IWA,SBH,SBD,SBA,WHH,WHD,WHA,country,league,season,SYH,SYD,SYA,B365H,B365D,B365A,GB>2.5,GB<2.5,GBAHH,GBAHA,GBAH,BWH,BWD,BWA,LBH,LBD,LBA,B365AHH,B365AHA,B365AH,SJH,SJD,SJA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,BSH,BSD,BSA,PSH,PSD,PSA,PSCH,PSCD,PSCA,HS,AS,HST,AST,HFKC,AFKC,HC,AC,HY,AY,HR,AR,Time,HF,AF,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Attendance,Referee,HHW,AHW,HO,AO,HBP,ABP,SOH,SOD,SOA,LBAHH,LBAHA,LBAH,LB,LB.1,LB.2,HT,AT
0,B1,20/05/01,Mechelen,Lierse,1.0,1.0,D,1.0,1.0,D,2.52,3.10,2.47,NaN,NaN,NaN,NaN,NaN,NaN,2.40,3.40,2.35,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B1,17/09/00,Germinal,Club Brugge,2.0,3.0,A,0.0,1.0,A,3.60,3.50,1.80,3.20,3.1,1.85,3.50,3.3,NaN,3.10,3.30,1.95,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B1,14/10/00,Beveren,Standard,0.0,3.0,A,0.0,2.0,A,3.30,3.40,1.85,3.40,3.2,1.75,3.20,3.4,NaN,3.25,3.45,1.85,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B1,06/09/00,Lokeren,Mechelen,3.0,2.0,H,0.0,2.0,A,1.66,3.50,3.95,1.55,3.3,4.50,1.67,3.5,NaN,1.70,3.50,3.80,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B1,27/08/00,Charleroi,Antwerp,1.0,0.0,H,1.0,0.0,H,1.60,3.35,4.80,1.60,3.3,4.00,1.62,3.5,NaN,1.60,3.40,4.50,belgium,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [11]:
df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)

bet_columns = ["B365", "BS", "BW", "GB", "IW", "LB", "PS", "SO", "SB", "SJ", "SY", "VC", "WH"]
home_bet = [col+"H" for col in bet_columns]
draw_bet = [col+"D" for col in bet_columns]
away_bet = [col+"A" for col in bet_columns]

def check_bet_columns(df):
    diff = set(home_bet) - set(df.columns)
    if len(diff)>0:
        print(f"There are {len(diff)} missing betting columns in the data.")
    else:
        print("All betting columns are present!")

df["AvgH"] = df[home_bet].mean(axis=1)
df["AvgD"] = df[draw_bet].mean(axis=1)
df["AvgA"] = df[away_bet].mean(axis=1)

check_bet_columns(df)

def dropnas(df):
    df = df.loc[~df["FTHG"].isna() & ~df["FTAG"].isna()].copy()
    df = df.loc[~df["HTHG"].isna() & ~df["HTAG"].isna()].copy()
    df = df.loc[~df["HomeTeam"].isna() & ~df["AwayTeam"].isna()].copy()
    df = df.loc[~df["AvgH"].isna() & ~df["AvgD"].isna() & ~df["AvgA"].isna()].copy()
    df = df.loc[~df["Div"].isna() & ~df["Date"].isna()].copy()
    df = df.loc[~df["FTR"].isna() & ~df["HTR"].isna()].copy()
    return df

def dropoutliers(df):
    df = df.loc[
        (df["FTHG"] >= 0) & (df["FTHG"] < 50) & (df["FTAG"] >= 0) & (df["FTAG"] < 50)
        ].copy()
    return df

def limit(val, lim):
    return val if val <= lim else lim

def capoutliers(df):
    df["ExtremH"] = df["AvgH"].apply(lambda x: x>10)
    df["AvgH"] = df["AvgH"].apply(lambda x: limit(x, 10))
    df["ExtremD"] = df["AvgD"].apply(lambda x: x>10)
    df["AvgD"] = df["AvgD"].apply(lambda x: limit(x, 10))
    df["ExtremA"] = df["AvgA"].apply(lambda x: x>10)
    df["AvgA"] = df["AvgA"].apply(lambda x: limit(x, 10))
    return df

def fill_goals(df):
    df["HTR"] = calculate_win(df["HTHG"], df["HTAG"])
    df["FTR_test"] = calculate_win(df["FTHG"], df["FTAG"])
    return df

def calculate_win(home_goals, away_goals):
    results = []
    for home, away in zip(home_goals, away_goals):
        if home < away:
            results.append("A")
        elif home > away:
            results.append("H")
        else:
            results.append("D")
    return results

def process_dates(df):
    df["Date"] = df["Date"].apply(
        lambda date: datetime.strptime(date, "%d/%m/%y") if len(date)==8 else datetime.strptime(date, "%d/%m/%Y")
    )
    return df

df = dropnas(df)
df = dropoutliers(df)
df = capoutliers(df)
df = fill_goals(df)
df = process_dates(df)

C:\Users\Martin\AppData\Local\Temp\ipykernel_23524\3291044820.py:1: DtypeWarning: Columns (65,76,91,154,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_data.csv", index_col=0).reset_index(drop=True)


All betting columns are present!


In [12]:
df["DivAndYear"] = df["Date"].apply(lambda x: str(x.year)) + df["Date"].apply(lambda x: " ") + df["Div"]

In [13]:
# Filter out old seasons and see impact on missing values
seasons = df["season"].unique()[-6:]
df = df.loc[df["season"].isin(seasons), :]

In [17]:
#Check NAs
pd.DataFrame((df.isna().sum()/df.shape[0]).sort_values(ascending=False)).T

,HBP,SOH,LBAH,LBAHA,SYH,SYD,SYA,BSH,SOD,GB>2.5,BSD,GB<2.5,GBAHH,GBAHA,GBAH,SJA,SJD,SJH,LB,BSA,GBH,SOA,AO,HO,AHW,HHW,AT,HT,LB.2,LB.1,B365AH,GBD,GBA,LBAHH,Attendance,ABP,SBH,SBD,SBA,AFKC,HFKC,LBH,LBD,LBA,Referee,WHCH,WHCA,WHCD,B365AHA,B365AHH,P<2.5,BWCD,BWCH,BWCA,P>2.5,PAHH,PAHA,IWCA,B365<2.5,IWCD,IWCH,B365>2.5,B365C>2.5,B365CAHH,B365CAHA,PC<2.5,B365CH,PCAHH,PCAHA,PC>2.5,AHh,B365C<2.5,VCCH,B365CD,B365CA,AvgCAHA,Avg>2.5,AvgCA,AHCh,MaxCA,MaxAHA,VCCD,Time,VCCA,AvgCH,MaxC>2.5,AvgAHA,AvgAHH,MaxAHH,MaxCAHH,Max>2.5,AvgCD,Max<2.5,MaxCH,MaxC<2.5,Avg<2.5,MaxA,MaxD,MaxH,AvgC>2.5,AvgC<2.5,MaxCAHA,MaxCD,AvgCAHH,BbMxH,Bb1X2,BbAv>2.5,BbOU,BbAvD,BbAvAHA,BbAH,BbAvAHH,BbAvH,BbMxAHA,BbMxD,BbAHh,BbMxA,BbAvA,BbMx>2.5,BbMx<2.5,BbAv<2.5,BbMxAHH,HF,AF,AR,HST,HY,HC,AST,HR,AY,AC,AS,HS,PSH,PSA,PSD,WHD,WHA,WHH,BWD,BWH,BWA,IWD,IWH,IWA,VCH,PSCH,VCA,VCD,B365D,B365A,PSCD,B365H,PSCA,ExtremH,ExtremD,ExtremA,FTR_test,Div,AvgA,AvgD,AvgH,Date,season,league,country,HTR,HTAG,HTHG,FTR,FTAG,FTHG,AwayTeam,HomeTeam,DivAndYear
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.968795,0.968795,0.663226,0.66284,0.662743,0.632215,0.513004,0.512738,0.512738,0.512013,0.511989,0.510901,0.510659,0.510659,0.510659,0.51049,0.510297,0.510055,0.509402,0.509257,0.509233,0.509137,0.509064,0.508702,0.508532,0.508532,0.508411,0.508339,0.508266,0.508242,0.508121,0.508097,0.508097,0.508073,0.508001,0.508001,0.507952,0.507952,0.507856,0.507856,0.507735,0.507711,0.507686,0.507686,0.507686,0.507686,0.507638,0.507638,0.507614,0.50759,0.507566,0.507541,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.507469,0.493184,0.493135,0.493135,0.49287,0.49287,0.492845,0.492821,0.492749,0.492749,0.4927,0.492531,0.492531,0.492531,0.492531,0.492531,0.492531,0.492531,0.492531,0.115102,0.115029,0.084187,0.084091,0.084018,0.083994,0.083728,0.083607,0.083607,0.083414,0.083414,0.083414,0.011602,0.01107,0.010949,0.008097,0.00788,0.007735,0.005535,0.005414,0.005221,0.003432,0.003191,0.003191,0.002635,0.00249,0.002417,0.002248,0.002055,0.002055,0.001958,0.001934,0.001837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Drop home/draw/away odds
df = df.drop(home_bet+draw_bet+away_bet, axis=1)

In [19]:
# Selected columns excluding all other odds data other than the avg ones one we calculated in the steps before
key_columns = ["country", "league", "season", "Div", "Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR",
              "Referee", "Attendance", "HS", "AS", "HST", "AST", "HHW", "AHW", "HC", "AC", "HF", "AF",
               "HFKC", "AFKC", "HO", "AO", "HY", "AY", "HR", "AR", "HBP", "ABP", "AvgH", "AvgD", "AvgA"]

In [20]:
# Select key columns
df = df.loc[:, key_columns].reset_index(drop=True)

In [27]:
# Calculate missing% for selected columns
missing_percentages = pd.DataFrame((df.isna().sum()/df.shape[0]).sort_values(ascending=False))

In [28]:
missing_percentages

,0
HO,1.000000
AO,1.000000
AHW,1.000000
HHW,1.000000
ABP,1.000000
HBP,1.000000
Attendance,1.000000
HFKC,0.968795
AFKC,0.968795
Referee,0.632215


In [30]:
# We drop the columns with a lot of missing percentages, as the values in them are not imputable in a meaningful way, and
# if we dropped the NA values from those columns, we would lose a big portion of our data
to_drop = missing_percentages[missing_percentages[0] > 0.2].index
to_drop

Index(['HO', 'AO', 'AHW', 'HHW', 'ABP', 'HBP', 'Attendance', 'HFKC', 'AFKC',
       'Referee', 'Time'],
      dtype='object')

In [35]:
df = df.drop(to_drop, axis=1).reset_index(drop=True)

In [36]:
missing_percentages = pd.DataFrame((df.isna().sum()/df.shape[0]).sort_values(ascending=False))
missing_percentages

,0
HF,0.115102
AF,0.115029
AR,0.084187
HST,0.084091
HY,0.084018
HC,0.083994
AST,0.083728
HR,0.083607
AY,0.083607
AS,0.083414


In [37]:
# We are gonna examine the fouls column to determine the type of missing data
df.loc[df["HF"].isna() | df["AF"].isna() ,:]

,country,league,season,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,AvgH,AvgD,AvgA
0,belgium,1,1617,B1,2017-02-04,Oostende,Mechelen,2.0,0.0,H,1.0,0.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.820000,3.710000,4.132857
1,belgium,1,1617,B1,2016-09-24,Charleroi,Oostende,2.0,1.0,H,1.0,0.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.126667,3.397143,3.364286
2,belgium,1,1617,B1,2016-08-05,Oostende,Club Brugge,1.0,0.0,H,0.0,0.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.965000,3.707143,1.851429
3,belgium,1,1617,B1,2016-12-17,Charleroi,Mouscron,2.0,0.0,H,1.0,0.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.470000,4.310000,6.772857
4,belgium,1,1617,B1,2016-08-27,Lokeren,Mouscron,2.0,1.0,H,1.0,0.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.821667,3.558571,4.245714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39705,turkey,1,1617,T1,2016-08-19,Adanaspor,Bursaspor,1.0,2.0,A,1.0,0.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.097143,3.518571,1.854286
39706,turkey,1,1617,T1,2016-09-10,Kayserispor,Galatasaray,1.0,1.0,D,1.0,1.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.140000,3.330000,2.242857
39707,turkey,1,1617,T1,2016-12-03,Buyuksehyr,Antalyaspor,2.0,2.0,D,1.0,1.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.517143,4.110000,6.088571
39708,turkey,1,1617,T1,2016-11-06,Osmanlispor,Kayserispor,1.0,1.0,D,0.0,0.0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.684286,3.647143,5.171429


In [44]:
# We can see there are red and yellow cards in the matches, which means there must've been some fauls on either side, we also checked
#the match statistics online on different pages and found that there were indeed fouls, but it's unkown why the data is missing, 
#so we just drop the values since there's only a few matches missing

# We also drop the matches with missing values in the other columnms since the percentage of missing values is very low
df_cleaned = df.dropna().reset_index(drop=True)
print("Dropped percentage of values: ", 1- df_cleaned.shape[0]/df.shape[0])



Dropped percentage of values:  0.1186067871990718


In [45]:
df_cleaned.isna().sum()

country     0
league      0
season      0
Div         0
Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
HS          0
AS          0
HST         0
AST         0
HC          0
AC          0
HF          0
AF          0
HY          0
AY          0
HR          0
AR          0
AvgH        0
AvgD        0
AvgA        0
dtype: int64

In [46]:
# Rename the columns
new_names = {"Div": "Division",
            "Date": "Date",
             "HomeTeam": "HomeTeam",
             "AwayTeam": "AwayTeam",
             "FTHG": "HomeGoals",
             "FTAG": "AwayGoals",
             "FTR": "Result",
             "HTHG": "HomeGoalsHalf",
             "HTAG": "AwayGoalsHalf",
             "HTR": "HalfResult",
             "HS": "HomeShots",
             "AS": "AwayShots",
             "HST": "HomeShotsTarget",
             "AST": "AwayShotsTarget",
             "HC": "HomeCorners",
             "AC": "AwayCorners",
             "HF": "HomeFouls",
             "AF": "AwayFouls",
             "HY": "HomeYellowCards",
             "AY": "AwayYellowCards",
             "HR": "HomeRedCards",
             "AR": "AwayRedCards",
             "AvgH": "AvgHomeOdds",
             "AvgD": "AvgDrawOdds",
             "AvgA": "AvgAwayOdds",
             "country": "Country",
             "league": "League",
             "season": "Season"
            }

In [47]:
df_cleaned.rename(columns=new_names, inplace=True)

In [48]:
df_cleaned.head()

,Country,League,Season,Division,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,HomeGoalsHalf,AwayGoalsHalf,HalfResult,HomeShots,AwayShots,HomeShotsTarget,AwayShotsTarget,HomeCorners,AwayCorners,HomeFouls,AwayFouls,HomeYellowCards,AwayYellowCards,HomeRedCards,AwayRedCards,AvgHomeOdds,AvgDrawOdds,AvgAwayOdds
0,belgium,1,1920,B1,2019-11-24,St Truiden,Charleroi,1.0,3.0,A,0.0,0.0,D,7.0,17.0,2.0,7.0,6.0,10.0,11.0,21.0,2.0,0.0,0.0,0.0,2.790000,3.405000,2.451667
1,belgium,1,1920,B1,2019-08-11,Mouscron,Gent,2.0,1.0,H,2.0,1.0,H,6.0,11.0,2.0,4.0,2.0,7.0,15.0,15.0,0.0,1.0,0.0,0.0,3.463333,3.685000,1.996667
2,belgium,1,1920,B1,2019-12-07,Cercle Brugge,Genk,1.0,2.0,A,0.0,1.0,A,16.0,10.0,9.0,5.0,3.0,3.0,13.0,13.0,1.0,3.0,0.0,0.0,4.598333,4.006667,1.685000
3,belgium,1,1920,B1,2019-12-08,Anderlecht,Charleroi,0.0,0.0,D,0.0,0.0,D,9.0,8.0,3.0,3.0,8.0,5.0,11.0,20.0,0.0,3.0,0.0,0.0,1.863333,3.628333,4.010000
4,belgium,1,1920,B1,2019-12-14,Charleroi,Cercle Brugge,3.0,0.0,H,1.0,0.0,H,8.0,7.0,7.0,1.0,4.0,3.0,12.0,18.0,1.0,1.0,0.0,0.0,1.431667,4.591667,6.928333


In [49]:
# Get the unique seasons
df_cleaned.loc[:, "Season"].unique()

array([1920, 2021, 2122, 1617, 1718, 1819], dtype=int64)

In [50]:
# Modify the season format for better data manipulation
df_cleaned["Season"] = df_cleaned["Season"].apply(lambda x: int(str(x)[:2]))

In [51]:
# Check new format
df_cleaned.loc[:, "Season"].unique()

array([19, 20, 21, 16, 17, 18], dtype=int64)

In [53]:
# Separate into train and test dataframe, train is 16-20 and test is season 21
train = df_cleaned.loc[df_cleaned["Season"] != 21, :]
test = df_cleaned.loc[df_cleaned["Season"] == 21, :]

In [54]:
train_seasons = np.sort(train["Season"].unique())
print(train_seasons)
test_seasons = np.sort(test["Season"].unique())
print(test_seasons)



[16 17 18 19 20]
[21]


### Feature engineering

In [55]:
stat_featuresH = ["HomeGoals", "HomeCorners", "HomeYellowCards"]
stat_featuresA = ["AwayGoals", "AwayCorners", "AwayYellowCards"]

def get_season_team_stats(data, stat_featuresH, stat_featuresA):
    grouped_home = train.groupby(["Season", "Division", "HomeTeam"])[stat_featuresH].agg(['sum', 'count']).reset_index()
    grouped_away = train.groupby(["Season", "Division", "AwayTeam"])[stat_featuresA].agg(['sum', 'count']).reset_index()
    grouped_home = grouped_home.rename(columns={"HomeTeam": "Team"})
    grouped_away = grouped_away.rename(columns={"AwayTeam": "Team"})
    
    merged_df = pd.merge(grouped_home, grouped_away, on=['Season', 'Division', 'Team'], suffixes=('_df1', '_df2'))

    merged_df["total_games"] = merged_df["HomeCorners"]["count"] + merged_df["AwayCorners"]["count"]
    merged_df["avg_goals"] = (merged_df["HomeGoals"]["sum"] + merged_df["AwayGoals"]["sum"])/merged_df["total_games"] 
    merged_df["avg_corners"] = (merged_df["HomeCorners"]["sum"] + merged_df["AwayCorners"]["sum"])/merged_df["total_games"]
    merged_df["avg_yellow"] = (merged_df["HomeYellowCards"]["sum"] + merged_df["AwayYellowCards"]["sum"])/merged_df["total_games"]
    merged_df.drop(columns=(stat_featuresH + stat_featuresA),level=0, inplace=True)
    merged_df.columns = [''.join(col).strip() if type(col) is tuple else col for col in merged_df.columns.values]

    return merged_df

season_statistics = get_season_team_stats(train, stat_featuresH, stat_featuresA).reset_index(drop=True) 
season_statistics

C:\Users\Martin\AppData\Local\Temp\ipykernel_23524\738316885.py:10: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_df = pd.merge(grouped_home, grouped_away, on=['Season', 'Division', 'Team'], suffixes=('_df1', '_df2'))


,Season,Division,Team,total_games,avg_goals,avg_corners,avg_yellow
0,16,D1,Augsburg,33,1.060606,3.060606,1.909091
1,16,D1,Bayern Munich,33,2.636364,7.151515,1.303030
2,16,D1,Darmstadt,32,0.875000,3.875000,1.843750
3,16,D1,Dortmund,31,1.935484,5.548387,1.645161
4,16,D1,Ein Frankfurt,34,1.058824,4.411765,2.411765
...,...,...,...,...,...,...,...
1606,20,T1,Konyaspor,39,1.205128,4.333333,2.256410
1607,20,T1,Rizespor,40,1.325000,3.625000,2.325000
1608,20,T1,Sivasspor,39,1.358974,5.051282,1.897436
1609,20,T1,Trabzonspor,38,1.236842,4.447368,1.921053


In [56]:
train.loc[:, "Prev_Season"] = train.loc[:, "Season"] - 1 
test.loc[:, "Prev_Season"] = test.loc[:, "Season"] - 1

C:\Users\Martin\AppData\Local\Temp\ipykernel_23524\3668212619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, "Prev_Season"] = train.loc[:, "Season"] - 1
C:\Users\Martin\AppData\Local\Temp\ipykernel_23524\3668212619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.loc[:, "Prev_Season"] = test.loc[:, "Season"] - 1


In [57]:
# Here we add the statistics from the previous season for both home and away team

def merge_stats_and_matches(match_data,stats_data):
    merged_df = pd.merge(match_data, stats_data, left_on = ["Prev_Season", "Division", "HomeTeam"],
                         right_on = ["Season", "Division", "Team"],
                         how="left",
                        suffixes=("","_home"))
    merged_df = merged_df.drop(["Season_home", "Team"], axis=1)
    merged_df = merged_df.rename(columns={"avg_goals": "avg_goals_home",
                                          "avg_corners": "avg_corners_home",
                                          "avg_yellow": "avg_yellow_home"})

    
    merged_df = pd.merge(merged_df, stats_data, left_on = ["Prev_Season", "Division", "AwayTeam"],
                         right_on = ["Season", "Division", "Team"],
                         how="left",
                        suffixes=("","_away"))
    merged_df = merged_df.drop(["Season_away", "Team"], axis=1)
    merged_df = merged_df.rename(columns={"avg_goals": "avg_goals_away",
                                          "avg_corners": "avg_corners_away",
                                          "avg_yellow": "avg_yellow_away"})
    
    return merged_df
train_merged = merge_stats_and_matches(train, season_statistics)
test_merged = merge_stats_and_matches(test, season_statistics)

### Random classifier performance

In [94]:
import random
# Random classifier
y_pred = [random.choice(["H", "A", "D"]) for _ in range(len(y_test))]

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)


Accuracy: 0.35
Classification Report:
               precision    recall  f1-score   support

           A       0.33      0.36      0.34      1387
           D       0.27      0.34      0.30      1179
           H       0.45      0.35      0.39      1910

    accuracy                           0.35      4476
   macro avg       0.35      0.35      0.34      4476
weighted avg       0.36      0.35      0.35      4476



### Baseline model using only odds

In [95]:
cols_x = ["AvgHomeOdds", "AvgDrawOdds", "AvgAwayOdds"]
cols_y = ["Result"]

train_merged.dropna(inplace=True)
test_merged.dropna(inplace=True)
X_train = train_merged[cols_x]
y_train = np.array(train_merged[cols_y]).ravel()
X_test = test_merged[cols_x]
y_test = np.array(test_merged[cols_y]).ravel()


In [96]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

Accuracy: 0.45
Classification Report:
               precision    recall  f1-score   support

           A       0.45      0.44      0.44      1387
           D       0.27      0.20      0.23      1179
           H       0.52      0.61      0.56      1910

    accuracy                           0.45      4476
   macro avg       0.41      0.42      0.41      4476
weighted avg       0.43      0.45      0.44      4476



### Adding some features to the model

In [97]:
cols_x = ["AvgHomeOdds", "AvgDrawOdds", "AvgAwayOdds"
          ,"avg_goals_home", "avg_corners_home", "avg_yellow_home",
          "avg_goals_away", "avg_corners_away", "avg_yellow_away"]
cols_y = ["Result"]


X_train = pd.get_dummies(train_merged, columns=['Division'])
X_test = pd.get_dummies(test_merged, columns=['Division']) 

dummies = X_train.columns[X_train.columns.str.startswith('Division_')]
cols_x = cols_x + dummies.values.tolist()
X_train = X_train[cols_x]
y_train = np.array(train_merged[cols_y]).ravel()
X_test = X_test[cols_x]
y_test = np.array(test_merged[cols_y]).ravel()


In [98]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

Accuracy: 0.49
Classification Report:
               precision    recall  f1-score   support

           A       0.48      0.47      0.47      1387
           D       0.31      0.12      0.17      1179
           H       0.53      0.73      0.61      1910

    accuracy                           0.49      4476
   macro avg       0.44      0.44      0.42      4476
weighted avg       0.45      0.49      0.45      4476



In [238]:
teams_df = pd.concat([
    train.rename(
        {"HomeTeam": "Team", "FTHG": "FTG", "HTHG": "HTG", "HS": "Shots", "HST": "ShotsOnTarget", "HHW": "HitWoodwork", "HC": "Corners", "HF": "Fouls", "HFKC": "KicksConceded", "HO": "Offsides", "HY": "YellowCards", "HR": "RedCards"}
        ,axis=1)[["Team", "FTG", "HTG", "Shots", "ShotsOnTarget", "HitWoodwork", "Corners", "Fouls", "KicksConceded", "Offsides", "YellowCards", "RedCards"]],
    train.rename(
        {"AwayTeam": "Team", "FTAG": "FTG", "HTAG": "HTG", "AS": "Shots", "AST": "ShotsOnTarget", "AHW": "HitWoodwork", "AC": "Corners", "AF": "Fouls", "AFKC": "KicksConceded", "AO": "Offsides", "AY": "YellowCards", "AR": "RedCards"}
        ,axis=1)[["Team", "FTG", "HTG", "Shots", "ShotsOnTarget", "HitWoodwork", "Corners", "Fouls", "KicksConceded", "Offsides", "YellowCards", "RedCards"]]
])

In [239]:
avg_shots_by_team = teams_df.groupby("Team")["Shots"].mean().to_dict()
avg_shots = teams_df["Shots"].mean()
for key, value in avg_shots_by_team.items():
    if pd.isna(value):
        avg_shots_by_team[key] = avg_shots

# encode teams into average shot count
for df_choose in (train, test, val):
    df_choose["HomeTeamEnc"] = df_choose["HomeTeam"].apply(lambda x: avg_shots_by_team.get(x, avg_shots))
    df_choose["AwayTeamEnc"] = df_choose["AwayTeam"].apply(lambda x: avg_shots_by_team.get(x, avg_shots))


In [240]:
cards_df = pd.concat([
    train.rename(
        {"HY": "YellowCards", "HR": "RedCards"}
        ,axis=1)[["Div", "YellowCards", "RedCards"]],
    train.rename(
        {"AY": "YellowCards", "AR": "RedCards"}
        ,axis=1)[["Div", "YellowCards", "RedCards"]]
])

In [241]:
yellow_cards_by_div = cards_df.groupby("Div")["YellowCards"].mean().to_dict()
red_cards_by_div = cards_df.groupby("Div")["RedCards"].mean().to_dict()

# encode divisions into average card given count
for df_choose in (train, test, val):
    df_choose["DivisionEncYellow"] = df_choose["Div"].map(yellow_cards_by_div)
    df_choose["DivisionEncRed"] = df_choose["Div"].map(red_cards_by_div)

In [242]:
chosen_columns = [
    "FTHG", "FTAG", "FTR",
    "HTHG", "HTAG", "HTR", 
    "HS", "AS", "HST", "AST",
    "HF", "AF",
    "HY", "AY", "HR", "AR", 
    "AvgH", "AvgA", "AvgD", 
    "HomeTeamEnc", "AwayTeamEnc"
    ]

In [243]:
train[chosen_columns].to_csv("train_processed.csv")
test[chosen_columns].to_csv("test_processed.csv")
val[chosen_columns].to_csv("val_processed.csv")